# Encoding High-Level Features: An Approach to Robust Transfer Learning.
## This code shows variational autoencoders being used to reduce dimensionality of feaure maps and provide the encoded representation to an image classifier. This work is an attempt to improve the robustness of the classic image classification architecture by performing a simple change to the overall system.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
def preprocess_data(image, label):

  image2 = tf.keras.applications.densenet.preprocess_input(image)
  return image2, label

In [ ]:
#### CREATE BASE MODEL FOR FEATURE EXTRACTION #####

def base_model():

  base_model = tf.keras.applications.DenseNet121(include_top= False, weights = 'imagenet', input_shape = (224,224,3))
  print(len(base_model.layers))
  for layer in base_model.layers[:200]:
    layer.trainable = False
  for layer in base_model.layers[200:]:
    layer.trainable = True
  base_model.summary()
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Lambda(lambda img: tf.image.resize(img, (224,224))))
  model.add(base_model)
  model.add(tf.keras.layers.GlobalAveragePooling2D())
  return model


In [ ]:
base_model = base_model()

29097984/29084464 [==============================] - 0s 0us/step
427
Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                   

In [ ]:
#### LOADING CIFAR10 DATASET

def load_data():
  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')

  # Convert class vectors to binary class matrices.
  y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
  y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
  train_ds = tf.data.Dataset.from_tensor_slices((x_train[:],y_train[:]))
  test_ds = tf.data.Dataset.from_tensor_slices((x_test[:2000], y_test[:2000]))
  train_ds = train_ds.map(lambda x,y: (preprocess_data(x,y)))
  test_ds = test_ds.map(lambda x,y: (preprocess_data(x,y)))
  batch_size = 100

  train_ds = train_ds.cache().batch(batch_size).prefetch(buffer_size=1000)
  test_ds = test_ds.cache().batch(batch_size).prefetch(buffer_size=1000)
  return train_ds, test_ds

train_ds, test_ds = load_data()

170508288/170498071 [==============================] - 2s 0us/step


## Let's start by training the classical image classifier composed of a feature extractor (DCNN) and a classifier (neural net).


In [ ]:
#### LOADING AND TRAINING THE COMPARATIVE MODEL
compare_model = tf.keras.Sequential()
compare_model.add(base_model)
compare_model.add(tf.keras.layers.Dropout(0.3))
compare_model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

In [ ]:
compare_model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

In [ ]:
training1 = compare_model.fit(train_ds, epochs = 30, verbose=1, validation_data = test_ds)

Epoch 1/30
500/500 [==============================] - 254s 462ms/step - loss: 0.3369 - accuracy: 0.8863 - val_loss: 0.3075 - val_accuracy: 0.8920
Epoch 2/30
500/500 [==============================] - 234s 468ms/step - loss: 0.1578 - accuracy: 0.9459 - val_loss: 0.2991 - val_accuracy: 0.9085
Epoch 3/30
500/500 [==============================] - 234s 468ms/step - loss: 0.1004 - accuracy: 0.9649 - val_loss: 0.2922 - val_accuracy: 0.9215
Epoch 4/30
500/500 [==============================] - 233s 466ms/step - loss: 0.0790 - accuracy: 0.9721 - val_loss: 0.3584 - val_accuracy: 0.9065
Epoch 5/30
500/500 [==============================] - 234s 467ms/step - loss: 0.0625 - accuracy: 0.9780 - val_loss: 0.2109 - val_accuracy: 0.9410
Epoch 6/30
500/500 [==============================] - 233s 467ms/step - loss: 0.0465 - accuracy: 0.9841 - val_loss: 0.3332 - val_accuracy: 0.9240
Epoch 7/30
500/500 [==============================] - 233s 466ms/step - loss: 0.0458 - accuracy: 0.9841 - val_loss: 0.2855 -

## We then freeze the DCNN to preserve the feature space and generate the feature maps for the dataset at hand.

In [ ]:
base_model.trainable = False
feat_maps_cifar = base_model.predict(train_ds, verbose = 1)

500/500 [==============================] - 134s 263ms/step


## Variational Feature Encoder
This section uses a variational autoencoder composed only of dense layers to perform dimensionality reduction of feature maps. It is trained to reconstruct feature maps from CIFAR10 dataset. Once it is trained we use the encoded representation in the latent space to feed a new classifier. This new system is now composed of the frozen DCNN, the encoder part of the variational autoencoder (VFE) and the classifier.

In [ ]:
#### CREATING THE VARIATIONAL AUTOENCODER

## THE SAMPLING BLOCK TAKES THE OUTPUT OF THE MEAN AND VARIANCE BLOCKS.
## IT THEN PROVIDES A DISTRIBUTION TO THE DECODER WHICH HAS THE TASK OF RECONSTRUCTING THE FEATURE MAPS.


class Sampling(tf.keras.layers.Layer):
  """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

  def call(self, inputs):
      z_mean, z_log_var = inputs
      batch = tf.shape(z_mean)[0]
      dim = tf.shape(z_mean)[1]
      epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
      return z_mean + tf.exp(0.5 * z_log_var) * epsilon

## DIMENSIONALITY OF THE LATENT SPACE
latent_dim = 256

encoder_inputs = tf.keras.Input(shape=(1024,))
x = tf.keras.layers.Dense(256, activation = 'relu')(encoder_inputs)
x = tf.keras.layers.BatchNormalization()(x)
z_mean = tf.keras.layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = tf.keras.layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = tf.keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

latent_inputs = tf.keras.Input(shape=(latent_dim,))
x = tf.keras.layers.Dense(512, activation='relu')(latent_inputs)
x = tf.keras.layers.BatchNormalization()(x)
decoder_outputs = tf.keras.layers.Dense(1024, activation="relu")(x)
decoder = tf.keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

optimizer = tf.keras.optimizers.Adam()

## DEFINING THE LOSSES TO BE USED IN THE TRAINING PROCESS
## MEAN SQUARRED ERROR
def mse_loss(y_true, y_pred):

    r_loss = tf.reduce_mean(tf.square(y_true - y_pred))
    ## MULTIPLICATIVE FACTOR ON THE TOTAL LOSS (1000)
    return 1000*r_loss

##Kullblack-Leibler Divergence Factor
def kl_loss(mean, log_var):

    kl_loss =  -0.5 * (1 + log_var - tf.square(mean) - tf.exp(log_var))
    ## BETA = 1
    return 1*kl_loss

## TOTAL LOSS
def vae_loss(y_true, y_pred, mean, var):

    r_loss = mse_loss(y_true, y_pred)
    kl_loss_ = kl_loss(mean, var)
    return  r_loss + kl_loss_



## DEFINING THE TRAINING PROCESS AND THE OBJECTIVE FUNCTION TO CALCULATE THE LOSS

class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):

        with tf.GradientTape() as tape:

            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = mse_loss(data, reconstruction)
            kl_loss_ = kl_loss(z_mean, z_log_var)
            total_loss = vae_loss(data, reconstruction, z_mean, z_log_var)

        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss_)

        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, 1024)]       0           []                               
                                                                                                  
 dense_12 (Dense)               (None, 256)          262400      ['input_11[0][0]']               
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 256)         1024        ['dense_12[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 z_mean (Dense)                 (None, 256)          65792       ['batch_normalization_3[0][

In [ ]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam(), metrics = ['accuracy'])
training_vae_cifar = vae.fit(feat_maps_cifar, epochs=2000, batch_size=128)

Epoch 1/2000
391/391 [==============================] - 2s 4ms/step - loss: 103.8051 - reconstruction_loss: 40.9204 - kl_loss: 2.6784
Epoch 2/2000
391/391 [==============================] - 2s 4ms/step - loss: 14.7844 - reconstruction_loss: 10.9683 - kl_loss: 2.6964
Epoch 3/2000
391/391 [==============================] - 2s 4ms/step - loss: 11.5390 - reconstruction_loss: 8.3653 - kl_loss: 2.6439
Epoch 4/2000
391/391 [==============================] - 2s 4ms/step - loss: 10.0894 - reconstruction_loss: 7.2095 - kl_loss: 2.6177
Epoch 5/2000
391/391 [==============================] - 2s 4ms/step - loss: 9.1049 - reconstruction_loss: 6.4170 - kl_loss: 2.5973
Epoch 6/2000
391/391 [==============================] - 2s 4ms/step - loss: 8.6451 - reconstruction_loss: 5.8684 - kl_loss: 2.5870
Epoch 7/2000
391/391 [==============================] - 1s 4ms/step - loss: 8.3293 - reconstruction_loss: 5.5687 - kl_loss: 2.5704
Epoch 8/2000
391/391 [==============================] - 2s 4ms/step - loss: 

KeyboardInterrupt: ignored

## Once the training of the VFE is done we freeze it and connect to the feature extractor to train a new classifier that will take encoded representations of feature maps and map it to given classes.

In [ ]:
vae.trainable = False

In [ ]:
def classifier():

  inputs = tf.keras.Input(shape = (256,))
  x = tf.keras.layers.Dropout(0.3)(inputs)
  outputs = tf.keras.layers.Dense(10, activation = 'softmax')(x)
  classifier = tf.keras.Model(inputs, outputs, name = 'classifier')
  return classifier

classifier = classifier()

In [ ]:
## CREATING THE IMAGE CLASSIFIER WITH VFE

model1 = tf.keras.Model(base_model.input, vae.encoder(base_model.output))
model_global = tf.keras.Model(model1.input, classifier(model1.output[2]))

In [ ]:
model1.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_input (InputLayer)   [(None, 32, 32, 3)]       0         
                                                                 
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 encoder (Functional)        [(None, 256),             395008    
                              (None, 256),                       
                              (None, 256)]                       
                                                           

In [ ]:
model_global.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_input (InputLayer)   [(None, 32, 32, 3)]       0         
                                                                 
 lambda (Lambda)             (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 encoder (Functional)        [(None, 256),             395008    
                              (None, 256),                       
                              (None, 256)]                       
                                                           

In [ ]:
model_global.compile(optimizer= 'Adam', loss='categorical_crossentropy', metrics=['accuracy'])
training = model_global.fit(train_ds, epochs = 50, verbose = 1, validation_data=test_ds)

Epoch 1/50
500/500 [==============================] - 139s 266ms/step - loss: 1.2806 - accuracy: 0.6280 - val_loss: 0.5026 - val_accuracy: 0.9200
Epoch 2/50
500/500 [==============================] - 137s 274ms/step - loss: 0.3975 - accuracy: 0.9309 - val_loss: 0.3017 - val_accuracy: 0.9365
Epoch 3/50
500/500 [==============================] - 138s 277ms/step - loss: 0.2781 - accuracy: 0.9452 - val_loss: 0.2415 - val_accuracy: 0.9375
Epoch 4/50
500/500 [==============================] - 140s 280ms/step - loss: 0.2246 - accuracy: 0.9482 - val_loss: 0.2156 - val_accuracy: 0.9410
Epoch 5/50
500/500 [==============================] - 140s 281ms/step - loss: 0.1951 - accuracy: 0.9525 - val_loss: 0.1981 - val_accuracy: 0.9405
Epoch 6/50
500/500 [==============================] - 141s 281ms/step - loss: 0.1787 - accuracy: 0.9529 - val_loss: 0.1883 - val_accuracy: 0.9405
Epoch 7/50
500/500 [==============================] - 141s 281ms/step - loss: 0.1645 - accuracy: 0.9540 - val_loss: 0.1841 -